In [1]:
# # Need to dive deeper to see how to add the package content in the context of uv
# # but for now:

# import sys
# from pathlib import Path

# # Add the directory containing your Lambda code to the path
# sys.path.append(str(Path("../cdk/lambda").resolve()))

In [ ]:
from lambda_function import 

In [2]:
# from lambda_function import 

In [3]:
## Few example files
import earthaccess
from datetime import datetime
import xarray as xr

collection_short_name = "MUR-JPL-L4-GLOB-v4.1"
drop_vars = ["dt_1km_data", "sst_anomaly"]
current_date = str(datetime.now().date()) + " 21:00:00"
granule_results = earthaccess.search_data(temporal=("2025-07-01 21:00:01", current_date), short_name=collection_short_name)

/home/jovyan/mursst-icechunk-updater/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
granule_results = granule_results[0:4]
granule_results

[Collection: {'Version': '4.1', 'ShortName': 'MUR-JPL-L4-GLOB-v4.1'}
 Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'BoundingRectangles': [{'WestBoundingCoordinate': -180, 'SouthBoundingCoordinate': -90, 'EastBoundingCoordinate': 180, 'NorthBoundingCoordinate': 90}]}}}
 Temporal coverage: {'RangeDateTime': {'EndingDateTime': '2025-07-02T21:00:00.000Z', 'BeginningDateTime': '2025-07-01T21:00:00.000Z'}}
 Size(MB): 702.3851881027222
 Data: ['https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/MUR-JPL-L4-GLOB-v4.1/20250702090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc'],
 Collection: {'Version': '4.1', 'ShortName': 'MUR-JPL-L4-GLOB-v4.1'}
 Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'BoundingRectangles': [{'WestBoundingCoordinate': -180, 'SouthBoundingCoordinate': -90, 'EastBoundingCoordinate': 180, 'NorthBoundingCoordinate': 90}]}}}
 Temporal coverage: {'RangeDateTime': {'EndingDateTime': '2025-07-03T21:00:00.000Z', 'BeginningDateTime

In [5]:
import icechunk as ic

def initial_icechunk_write(store_url:str) -> ic.Repository:
    pass

import random
from urllib.parse import urlparse, urlunparse
import os

def get_prefix_from_url(url:str) -> str:
    parsed = urlparse(url)
    path_without_file = os.path.dirname(parsed.path)
    new_parsed = parsed._replace(path=path_without_file)
    prefix = urlunparse(new_parsed)+'/' # we always need a trailing slash for ic (see https://github.com/earth-mover/icechunk/issues/1100)
    return prefix
    

def dummy_github_run_id():
    # GitHub run IDs are currently around 10 digits (e.g., 6768204927)
    # This generates a dummy 10-digit integer in a similar range
    return str(random.randint(10**9, 10**10 - 1))

from icechunk import S3StaticCredentials

def get_icechunk_creds(daac: str = None) -> S3StaticCredentials:
    if daac is None:
        daac = "PODAAC"  # TODO: Might want to change this for a more general version
        # https://github.com/nsidc/earthaccess/discussions/1051 could help here.
    # assumes that username and password are available in the environment
    # TODO: accomodate rc file?
    auth = earthaccess.login(strategy="environment")
    if not auth.authenticated:
        raise PermissionError("Could not authenticate using environment variables")
    creds = auth.get_s3_credentials(daac=daac)
    return S3StaticCredentials(
        access_key_id=creds["accessKeyId"],
        secret_access_key=creds["secretAccessKey"],
        expires_after=datetime.fromisoformat(creds["expiration"]),
        session_token=creds["sessionToken"],
    )


store_bucket_url = f"s3://nasa-veda-scratch/jbusecke/icechunk/mursst-testing/run_id_{dummy_github_run_id()}"
store_bucket_url_parsed = urlparse(store_bucket_url)

storage = ic.s3_storage(
    bucket = store_bucket_url_parsed.netloc,
    prefix = store_bucket_url_parsed.path,
    from_env=True,
)

direct_access_urls = [g.data_links(access='direct')[0] for g in granule_results]
target_url = direct_access_urls[0]

config = ic.RepositoryConfig.default()
config.set_virtual_chunk_container(
    ic.VirtualChunkContainer(
        get_prefix_from_url(target_url),
        ic.s3_store(region='us-west-2')
    )
)

virtual_credentials = ic.containers_credentials(
    {
        get_prefix_from_url(target_url): ic.s3_refreshable_credentials(
                get_credentials=get_icechunk_creds
            )
    }
)
    
repo = ic.Repository.create(
    storage=storage,
    config=config,
    authorize_virtual_chunk_access=virtual_credentials,
)
print(store_bucket_url)
print(repo.config.virtual_chunk_containers)

s3://nasa-veda-scratch/jbusecke/icechunk/mursst-testing/run_id_5866677505
{'s3://podaac-ops-cumulus-protected/MUR-JPL-L4-GLOB-v4.1/': <builtins.VirtualChunkContainer object at 0x7f96e9166160>}


In [6]:
target_url

's3://podaac-ops-cumulus-protected/MUR-JPL-L4-GLOB-v4.1/20250702090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc'

In [15]:
# now for the vz component
import virtualizarr
print(f"VZ version:{virtualizarr.__version__}")

from typing import TYPE_CHECKING
if TYPE_CHECKING:
    from obstore.store import S3Credential

from obstore.store import S3Store
from virtualizarr import open_virtual_mfdataset
from virtualizarr.parsers import HDFParser
from virtualizarr.registry import ObjectStoreRegistry
from typing import Tuple

import earthaccess

#TODO: I should probably use the EDL authenticator that comes with obstore
# def get_obstore_credentials() -> S3Credential:# does not work properly in a notebook
def get_obstore_credentials(): # quick and dirty fix
    auth = earthaccess.login()
    creds = auth.get_s3_credentials(daac = "PODAAC")
    return {
        "access_key_id": creds['accessKeyId'],
        "secret_access_key": creds['secretAccessKey'],
        "token": creds['sessionToken'],
        "expires_at": datetime.fromisoformat(creds['expiration']),
    }

# from obstore.auth.earthdata import NasaEarthdataCredentialProvider
# cp = NasaEarthdataCredentialProvider(credentials_url)

def store_and_registry_from_url(url:str) -> Tuple[S3Store, ObjectStoreRegistry]:
    parsed = urlparse(url)
    parsed_wo_path = parsed._replace(path="")
    bucket = parsed.netloc
    print(bucket)
    cp = get_obstore_credentials
    store = S3Store(
        bucket=bucket,
        region="us-west-2",
        credential_provider=cp
    )
    registry = ObjectStoreRegistry({parsed_wo_path.geturl():store})
    return store, registry
    
store, registry = store_and_registry_from_url(target_url)
parser = HDFParser()

def preprocess(ds:xr.Dataset) -> xr.Dataset:
    return ds.drop_vars(drop_vars, errors="ignore")

vds = open_virtual_mfdataset(
    direct_access_urls,
    registry=registry,
    parser=parser,
    # decode_timedelta=True,
    preprocess=preprocess,
    # parallel="lithops" #TODO: Investigate why this fails? Is that an issue with uv?
    concat_dim="time",
    combine="nested",
    data_vars="minimal",
    coords="minimal",
    compat="override",
    join="exact",
    loadable_variables=['time'],
)
vds

VZ version:2.0.0
podaac-ops-cumulus-protected


/home/jovyan/mursst-icechunk-updater/.venv/lib/python3.12/site-packages/virtualizarr/xarray.py:341: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta64 values. To continue decoding timedeltas based on the presence of a timedelta-like units attribute, users will need to explicitly opt-in by passing True or CFTimedeltaCoder(decode_via_units=True) to decode_timedelta. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  with xr.open_zarr(
/home/jovyan/mursst-icechunk-updater/.venv/lib/python3.12/site-packages/numcodecs/zarr3.py:145: UserWarning: Numcodecs codecs are not in the Zarr version 3 specification and may not be supported by other zarr implementations.
  super().__init__(**codec_config)
/home/jovyan/mursst-icechunk-up

<xarray.Dataset> Size: 16GB
Dimensions:           (time: 4, lat: 17999, lon: 36000)
Coordinates:
  * time              (time) datetime64[ns] 32B 2025-07-02T09:00:00 ... 2025-...
    lat               (lat) float32 72kB ManifestArray<shape=(17999,), dtype=...
    lon               (lon) float32 144kB ManifestArray<shape=(36000,), dtype...
Data variables:
    analysed_sst      (time, lat, lon) int16 5GB ManifestArray<shape=(4, 1799...
    analysis_error    (time, lat, lon) int16 5GB ManifestArray<shape=(4, 1799...
    mask              (time, lat, lon) int8 3GB ManifestArray<shape=(4, 17999...
    sea_ice_fraction  (time, lat, lon) int8 3GB ManifestArray<shape=(4, 17999...
Attributes: (12/47)
    Conventions:                CF-1.7
    title:                      Daily MUR SST, Final product
    summary:                    A merged, multi-sensor L4 Foundation SST anal...
    references:                 http://podaac.jpl.nasa.gov/Multi-scale_Ultra-...
    institution:                Jet Propulsion Laboratory
    history:                    created at nominal 4-day latency; replaced nr...
    ...                         ...
    project:                    NASA Making Earth Science Data Records for Us...
    publisher_name:             GHRSST Project Office
    publisher_url:              http://www.ghrsst.org
    publisher_email:            ghrsst-po@nceo.ac.uk
    processing_level:           L4
    cdm_data_type:              grid

In [16]:
vds.isel(time=slice(0,3))

ValueError: Invalid indexer for array. Indexer length must be less than or equal to the number of dimensions in the array, but indexer=(slice(0, 3, None), slice(None, None, None), slice(None, None, None), Ellipsis) has length 4 and array has 3 dimensions.
If concatenating using xarray, ensure all non-coordinate data variables to be concatenated include the concatenation dimension, or consider passing `data_vars='minimal'` and `coords='minimal'` to the xarray combining function.

In [8]:
session = repo.writable_session('main')
vds.vz.to_icechunk(session.store)

In [9]:
session.commit('Initial Data')

'TP90RC7BNPPM05ZM0HW0'

In [10]:
# reread
import xarray as xr
rsession = repo.readonly_session('main')
ds_saved = xr.open_zarr(rsession.store, zarr_format=3, consolidated=False)
ds_saved

/home/jovyan/mursst-icechunk-updater/.venv/lib/python3.12/site-packages/numcodecs/zarr3.py:145: UserWarning: Numcodecs codecs are not in the Zarr version 3 specification and may not be supported by other zarr implementations.
  super().__init__(**codec_config)


<xarray.Dataset> Size: 73GB
Dimensions:           (time: 4, lat: 17999, lon: 36000)
Coordinates:
  * lat               (lat) float32 72kB -89.99 -89.98 -89.97 ... 89.98 89.99
  * lon               (lon) float32 144kB -180.0 -180.0 -180.0 ... 180.0 180.0
  * time              (time) datetime64[ns] 32B 2025-07-02T09:00:00 ... 2025-...
Data variables:
    analysed_sst      (time, lat, lon) float64 21GB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    analysis_error    (time, lat, lon) float64 21GB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    mask              (time, lat, lon) float32 10GB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float64 21GB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
Attributes: (12/47)
    Conventions:                CF-1.7
    title:                      Daily MUR SST, Final product
    summary:                    A merged, multi-sensor L4 Foundation SST anal...
    references:                 http://podaac.jpl.nasa.gov/Multi-scale_Ultra-...
    institution:                Jet Propulsion Laboratory
    history:                    created at nominal 4-day latency; replaced nr...
    ...                         ...
    project:                    NASA Making Earth Science Data Records for Us...
    publisher_name:             GHRSST Project Office
    publisher_url:              http://www.ghrsst.org
    publisher_email:            ghrsst-po@nceo.ac.uk
    processing_level:           L4
    cdm_data_type:              grid

In [46]:
dt = ds_saved.time.data[3]
py_dt = dt.astype('datetime64[ms]').astype(datetime)
py_dt

datetime.datetime(2025, 7, 5, 9, 0)

In [42]:
from datetime import datetime, timedelta

In [44]:
py_dt + timedelta(seconds=1)

datetime.datetime(2025, 7, 5, 9, 0, 1)

In [ ]:
dt = ds_saved.time[3].data.flatten()[0]

In [47]:
a = [0,2,3]

In [48]:
a[:1]

[0]

In [12]:
# is this info available on a reloaded copy?

In [13]:
## Exploring persisting the config (later)
Does not seem to be necessary!

SyntaxError: invalid syntax (1721281668.py, line 2)

In [24]:
import fsspec
fs = fsspec.filesystem('s3')
fs.ls("s3://nasa-eodc-public/icechunk/")

['nasa-eodc-public/icechunk/MUR-JPL-L4-GLOB-v4.1-virtual-v1',
 'nasa-eodc-public/icechunk/MUR-JPL-L4-GLOB-v4.1-virtual-v1-p2']

In [2]:
## Check out the starting date for the second store:
from lambda_function import (
    open_icechunk_repo, 
    get_icechunk_storage, 
    open_xr_dataset_from_branch,
    get_prefix_from_url, 
    example_target_url,
    get_container_credentials,
)
import icechunk

# url = "s3://nasa-eodc-public/icechunk/MUR-JPL-L4-GLOB-v4.1-virtual-v1"
# url = "s3://nasa-veda-scratch/jbusecke/icechunk/mursst-testing/MUR-JPL-L4-GLOB-v4.1-virtual-v2-p2"
url = "s3://nasa-veda-scratch/jbusecke/icechunk/mursst-testing/MUR_test_9"
# storage = get_icechunk_storage("s3://nasa-eodc-public/icechunk/MUR-JPL-L4-GLOB-v4.1-virtual-v1-p2")
storage = get_icechunk_storage(url)

# repo = open_icechunk_repo(storage)
config = icechunk.RepositoryConfig.default()
config.set_virtual_chunk_container(
    icechunk.VirtualChunkContainer(
        get_prefix_from_url(
            example_target_url
        ),  # TODO: Again I dont like this example url here...
        icechunk.s3_store(region="us-west-2"),
    )
)
repo = icechunk.Repository.open(
    storage=storage,
    config=config,
    authorize_virtual_chunk_access=get_container_credentials(example_target_url),
)
session = repo.readonly_session('main')
import xarray as xr
ds = xr.open_zarr(session.store, zarr_format=3, consolidated=False)
ds

Defining icechunk storage for s3


/home/jovyan/mursst-icechunk-updater/.venv/lib/python3.12/site-packages/numcodecs/zarr3.py:145: UserWarning: Numcodecs codecs are not in the Zarr version 3 specification and may not be supported by other zarr implementations.
  super().__init__(**codec_config)


<xarray.Dataset> Size: 4TB
Dimensions:           (time: 212, lat: 17999, lon: 36000)
Coordinates:
  * lon               (lon) float32 144kB -180.0 -180.0 -180.0 ... 180.0 180.0
  * lat               (lat) float32 72kB -89.99 -89.98 -89.97 ... 89.98 89.99
  * time              (time) datetime64[ns] 2kB 2025-01-01T09:00:00 ... 2025-...
Data variables:
    analysed_sst      (time, lat, lon) float64 1TB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    analysis_error    (time, lat, lon) float64 1TB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    mask              (time, lat, lon) float32 549GB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float64 1TB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
Attributes: (12/47)
    Conventions:                CF-1.7
    title:                      Daily MUR SST, Final product
    summary:                    A merged, multi-sensor L4 Foundation SST anal...
    references:                 http://podaac.jpl.nasa.gov/Multi-scale_Ultra-...
    institution:                Jet Propulsion Laboratory
    history:                    created at nominal 4-day latency; replaced nr...
    ...                         ...
    project:                    NASA Making Earth Science Data Records for Us...
    publisher_name:             GHRSST Project Office
    publisher_url:              http://www.ghrsst.org
    publisher_email:            ghrsst-po@nceo.ac.uk
    processing_level:           L4
    cdm_data_type:              grid

In [23]:
repo.config

RepositoryConfig(inline_chunk_threshold_bytes=None, get_partial_values_concurrency=None, compression=None, caching=None, storage=None, manifest=None)